In [19]:
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

In [20]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_16 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [21]:
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input, output_tensor)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_5:0", shape=(?, 64), dtype=float32) at layer "input_5". The following previous layers were accessed without issue: []

In [22]:
import numpy as np

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 1s 703us/step - loss: 11.6803
Epoch 2/10
1000/1000 [==============================] - 0s 26us/step - loss: 11.5243
Epoch 3/10
1000/1000 [==============================] - 0s 26us/step - loss: 11.4993
Epoch 4/10
1000/1000 [==============================] - 0s 26us/step - loss: 11.4853
Epoch 5/10
1000/1000 [==============================] - 0s 27us/step - loss: 11.4755
Epoch 6/10
1000/1000 [==============================] - 0s 27us/step - loss: 11.4681
Epoch 7/10
1000/1000 [==============================] - 0s 26us/step - loss: 11.4632
Epoch 8/10
1000/1000 [==============================] - 0s 26us/step - loss: 11.4581
Epoch 9/10
1000/1000 [==============================] - ETA: 0s - loss: 11.32 - 0s 26us/step - loss: 11.4551
Epoch 10/10
1000/1000 [==============================] - 0s 194us/step


2개의 입력을 가진 질문-응답 모델의 함수형 API 구현

In [23]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss= 'categorical_crossentropy', metrics=['acc'])

In [24]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answers = to_categorical(answers)
model.fit([text, question], answers, epochs=10, batch_size=128)
model.fit({'text' : text, 'question' : question}, answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 4s 4ms/step - loss: 6.2144 - acc: 0.0020
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1989 - acc: 0.0410
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1594 - acc: 0.0170
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.0778 - acc: 0.0110
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.9849 - acc: 0.0130
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.8933 - acc: 0.0130
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.8109 - acc: 0.0150
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.7354 - acc: 0.0230
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.6742 - acc: 0.0260
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.6080 - acc: 0.0300
Epoch 1/10
1000/1000 [=======

In [25]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(question_vocabulary_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)

gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

다중 출력 모델의 컴파일 옵션: 다중 손실

In [26]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop', loss={
    'age' : 'mse', 
    'income' : 'categorical_crossentropy', 
    'gender' : 'binary_crossentropy'})

다중 출력 모델의 컴파일 옵션: 손실 가중치

In [27]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop', loss={
    'age' : 'mse', 
    'income' : 'categorical_crossentropy', 
    'gender' : 'binary_crossentropy'},
             loss_weights={ 'age' : 0.25, 
    'income' : 1., 
    'gender' : 10.})

In [ ]:
model.fit(posts_input, [age_targets, income_targets, gender_targets], epochs=10, batch_size = 64)

model.fit(
    posts_input, {'age' : age_targets, 'income' : income_targets, 'gender' : gender_targets},
    epochs=10,
    batch_size = 64)

인셉션 모듈의 예

In [ ]:
from keras import layers

branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

잔차 연결의 예

In [ ]:
x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x]) #= layers.Add()([y, x])

특성 맵의 크기가 다른 경우 선형 변환을 사용하여 잔차 연결 구한 예

In [ ]:
x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual]) #= layers.Add()([y, x])

함수형 API로 재사용 층을 사용하는 모델 구현의 예

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

샴 비전 모델을 구현하는 예

In [ ]:
from keras import layers
from keras import applications
from keras.models import Input

xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250,250,3))
right_input = Input(shape=(250,250,3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)

EarlyStopping 콜백 모델의 예시

In [ ]:
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(
    monitor='val_acc',
    patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
    filepath='my_model.h5',
    monitor='val_loss',
    save_best_only=True,
    )
]

moel.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['acc'])

model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

ReduceLROnPlateau 콜백 모델의 예시

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=10,
    )
]

model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

자작 콜백 예시

In [ ]:
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')    
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activation_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, activationsvations)
        f.close()

텐서보드를 사용한 텍스트 분류 모델

In [31]:
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [37]:
callbacks = [
    keras.callbacks.TensorBoard(
    log_dir='C:/Users/J/Desktop/my_log_dir',
    histogram_freq = 1,
    embeddings_freq = 1,
    embeddings_data = x_train[:100],
    #embedding_data = x_test
    )
]
   
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 2s 107us/step - loss: 0.3151 - acc: 0.7245 - val_loss: 0.5311 - val_acc: 0.6966
Epoch 2/20
20000/20000 [==============================] - 1s 67us/step - loss: 0.2589 - acc: 0.6907 - val_loss: 0.9869 - val_acc: 0.4968
Epoch 3/20
20000/20000 [==============================] - 1s 69us/step - loss: 0.2132 - acc: 0.6180 - val_loss: 0.7945 - val_acc: 0.5438
Epoch 4/20
20000/20000 [==============================] - 1s 67us/step - loss: 0.1814 - acc: 0.5609 - val_loss: 0.7579 - val_acc: 0.5132
Epoch 5/20
20000/20000 [==============================] - 1s 68us/step - loss: 0.1401 - acc: 0.5556 - val_loss: 0.9508 - val_acc: 0.4184
Epoch 6/20
20000/20000 [==============================] - 1s 67us/step - loss: 0.1398 - acc: 0.4384 - val_loss: 0.9757 - val_acc: 0.3770
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch 7/20
20000/20000 [====================

In [54]:
from keras.utils import plot_model

plot_model(model, to_file='model.png')
plot_model(model, show_shapes = True, to_file='model.png')

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.